In [5]:
import torch
import os
from torchvision.models import inception_v3
from torchmetrics.image.fid import FrechetInceptionDistance
from torchvision.transforms import functional as TF
from PIL import Image
from tqdm import tqdm

device = torch.device("cuda")

# Function to load and preprocess images from a directory
def load_images_from_directory(directory, expected_count=30000):
    images = []
    image_count = 0

    for filename in tqdm(os.listdir(directory), desc=f"Loading images from {directory}"):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Check for image files
            image_count += 1
            img = Image.open(os.path.join(directory, filename)).convert('RGB')
            img = TF.resize(img, [299, 299])  # Resize image
            img = TF.to_tensor(img)  # Convert to tensor
            img = TF.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
            images.append(img.unsqueeze(0))  # Add batch dimension

    if image_count != expected_count:
        raise ValueError(f"Expected {expected_count} images in directory '{directory}', but found {image_count} images.")

    return torch.cat(images, dim=0)  # Concatenate all images into a single tensor

# Load the Inception V3 model
model = inception_v3(pretrained=True)
model.fc = torch.nn.Identity()  # Remove the final fully connected layer
model.eval().to(device)

# Directories containing your images
real_images_dir = '/weka/proj-fmri/shared/coco/sampled_imgs'
generated_images_dir = '/weka/proj-fmri/shared/coco/vd_imgs'

# Load images
real_images = load_images_from_directory(real_images_dir).to(device)
generated_images = load_images_from_directory(generated_images_dir).to(device)

# Extract features
with torch.no_grad():  # Disable gradient calculation for efficiency
    real_features = model(real_images)
    generated_features = model(generated_images)

# Initialize FID
fid = FrechetInceptionDistance(feature=2048)

# Update state with real and generated features
fid.update(real_features, real=True)
fid.update(generated_features, real=False)

# Compute FID score
fid_score = fid.compute()
print(f'FID score: {fid_score}')


Loading images from /weka/proj-fmri/shared/coco/sampled_imgs: 100%|█████████████████████████████████████████████████████████████████████████████████| 30000/30000 [04:36<00:00, 108.42it/s]
Loading images from /weka/proj-fmri/shared/coco/vd_imgs: 100%|███████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [56:04<00:00,  8.92it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 29.97 GiB. GPU 0 has a total capacty of 39.56 GiB of which 9.07 GiB is free. Including non-PyTorch memory, this process has 30.49 GiB memory in use. Of the allocated memory 30.07 GiB is allocated by PyTorch, and 21.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
import subprocess
import yaml

# Run 'pip list' command and capture the output
pip_list_output = subprocess.check_output(["pip", "list"]).decode("utf-8")

# Split the output into lines and ignore the header
package_lines = pip_list_output.strip().split('\n')[2:]

# Create a list of dictionaries for each package
packages = []
for line in package_lines:
    package_info = line.strip().split()
    package_name = package_info[0]
    package_version = package_info[1]
    packages.append({"package": package_name, "version": package_version})

# Define the path for the output YAML file
output_yaml_file = "pip_list.yaml"

# Write the package information to the YAML file
with open(output_yaml_file, "w") as yaml_file:
    yaml.dump(packages, yaml_file, default_flow_style=False)

print(f"Pip list has been saved to {output_yaml_file}")


Pip list has been saved to pip_list.yaml
